In [1]:
import csv
import itertools
import os
import re
from collections import Counter
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.tslib import Timestamp

In [2]:
%matplotlib inline
matplotlib.rcParams['figure.max_open_warning'] = 100
matplotlib.rcParams['lines.linewidth'] = 1.0

In [3]:
from IPython.display import display

def print_full_df(df):
    pd.set_option('display.max_rows', len(df))
    display(df)
    pd.reset_option('display.max_rows')

# Grupos

## Socios activos 2016

In [4]:
# 1.       CTA: categoría para niños y jóvenes hasta bachillerato
# 2.       CTI: categoría infantil, hasta doce años y solo es para Bogotá (blaa –biblioteca Luis Ángel Arango)
# 3.       CTB: público general (solo permite préstamo de libros)
# 4.       CTC: público en general (permite préstamo de libros y material audiovisual)
# 5.       CTF: categoría familiar, pueden acceder a los beneficios hasta tres personas del grupo familiar que se asocia
# 6.       CE1, CE2, CE2: categorías empresariales (solo para instituciones)
# 7.       CIS: categoría Inclusión Social
# 8.       CTU: categoría Universitaria
# 9.       VIP: Investigadores, préstamo de 30 materiales (libros y material audiovisual)

#      Categoría de lector: CE1 Categoría empresarial                                   218
#      Categoría de lector: CE2 Categoría empresarial 2                                   5
#      Categoría de lector: CEM Categoría Empleado/Pensionado                          1593
#      Categoría de lector: CIN Categoría Interbibliotecario                            128
#      Categoría de lector: CIP Categoría Investigador                                  351
#      Categoría de lector: CIS Categoría Inclusión Preferencial                      15974
#      Categoría de lector: CIV Categoría Investigadores                                 72
#      Categoría de lector: CML Categoría Mediador de Proyectos Culturales                2
#      Categoría de lector: CSU Categoría Sucursal                                       91
#      Categoría de lector: CTA Categoría Básica A                                    10610
#      Categoría de lector: CTB Categoría B                                            1359
#      Categoría de lector: CTC Categoría C                                             224
#      Categoría de lector: CTF Categoría familiar                                     1105
#      Categoría de lector: CTI Categoría Infantil                                     1004
#      Categoría de lector: CTU Categoría General                                     17402
#      Categoría de lector: PTE PROCESOS TÉCNICOS                                        95

In [5]:
%%time

dtypes = {
    'Sucursal': str,
    'Tratamiento': str,
    'Barrio': str,
    'Ciudad': str,
    'CATEG': str,
    'Ocupación1': str,
    'Nivel Estudio1': str,
    'Área de Estudio1': str,
    'Edad': 'category',
}

perfiles_df = pd.read_csv(
    'valores_grupos_2016.csv', header=0, dtype=dtypes)
# perfiles_df = perfiles_df.fillna('')

CPU times: user 64.6 ms, sys: 16.1 ms, total: 80.8 ms
Wall time: 79.8 ms


In [6]:
perfiles_df.shape

(50272, 9)

In [7]:
perfiles_df.head()

,Sucursal,Tratamiento,Barrio,Ciudad,CATEG,Ocupación1,Nivel Estudio1,Área de Estudio1,Edad
0,BLAA,OTRO,chapinero,bogota,CIN,universidad,OTRO,OTRO,OTRO
1,BLAA,sra.,OTRO,bogota,CEM,pensionado,OTRO,OTRO,50-64
2,BLAA,sra.,icata,bogota,CEM,empleado,profesional,filosofía y letras,OTRO
3,ARM,sr.,OTRO,OTRO,CEM,empleado,profesional,economía,50-64
4,BLAA,sr.,soledad,bogota,CEM,OTRO,OTRO,OTRO,OTRO


# Eliminando valores en blanco

In [8]:
perfiles_df = perfiles_df[['Ciudad', 'Barrio', 'Nivel Estudio1', 'Área de Estudio1']]

In [9]:
perfiles_df = perfiles_df[
    (perfiles_df['Ciudad'] != 'OTRO') &
    (perfiles_df['Barrio'] != 'OTRO')
]

In [10]:
perfiles_df.shape

(41904, 4)

In [11]:
perfiles_df.head()

,Ciudad,Barrio,Nivel Estudio1,Área de Estudio1
0,bogota,chapinero,OTRO,OTRO
2,bogota,icata,profesional,filosofía y letras
4,bogota,soledad,OTRO,OTRO
5,bogota,galerias,OTRO,OTRO
7,bogota,velodromo,OTRO,OTRO


# Agrupando

## Todas las ciudades

In [12]:
ciudades_sucursales = {
    'armenia': 'ARM',
    'barranquilla': 'BAR',
    'bogota': 'BLAA',
    'bucaramanga': 'BUC',
    'buenaventura': 'BUE',
    'cali': 'CAL',
    'cartagena': 'CAR',
    'cucuta': 'CUC',
    'florencia': 'FLO',
    'girardot': 'GIR',
    'honda': 'HON',
    'ibague': 'IBA',
    'ipiales': 'IPI',
    'leticia': 'LET',
    'manizales': 'MAN',
    'medellin': 'MED',
    'monteria': 'MON',
    'neiva': 'NEI',
    'pasto': 'PAS',
    'pereira': 'PER',
    'popayan': 'POP',
    'quibdo': 'QUI',
    'riohacha': 'RIO',
    'san andres': 'SAN',
    'santa marta': 'STM',
    'sincelejo': 'SIN',
    'tunja': 'TUN',
    'valledupar': 'VAL',
    'villavicencio': 'VIL',
}

### Top 1 nivel de estudios por ciudad

In [13]:
niveles_ciudades = perfiles_df[perfiles_df['Nivel Estudio1'] != 'OTRO'] \
    .groupby(['Ciudad'])['Nivel Estudio1'] \
    .agg(lambda x: x.value_counts().index[0])

In [14]:
top_niveles_ciudades_ls = [
    (x, y)
    for x, y in niveles_ciudades.iteritems()
    if x in ciudades_sucursales
]

In [15]:
top_niveles_ciudades_df = pd.DataFrame(top_niveles_ciudades_ls, columns=['Ciudad', 'Top nivel de estudios'])

In [16]:
print_full_df(top_niveles_ciudades_df)

,Ciudad,Top nivel de estudios
0,armenia,profesional
1,barranquilla,universitario 2 nivel
2,bogota,universitario 1 nivel
3,bucaramanga,profesional
4,buenaventura,básica primaria
5,cali,profesional
6,cartagena,universitario 2 nivel
7,cucuta,universitario 1 nivel
8,florencia,básica primaria
9,girardot,secundaria


### Top 1 área de estudios por ciudad

In [17]:
areas_ciudades = perfiles_df[perfiles_df['Área de Estudio1'] != 'OTRO'] \
    .groupby(['Ciudad'])['Área de Estudio1'] \
    .agg(lambda x: x.value_counts().index[0])

In [18]:
top_areas_ciudades_ls = [
    (x, y)
    for x, y in areas_ciudades.iteritems()
    if x in ciudades_sucursales
]

In [19]:
top_areas_ciudades_df = pd.DataFrame(top_areas_ciudades_ls, columns=['Ciudad', 'Top área de estudios'])

In [20]:
print_full_df(top_areas_ciudades_df)

,Ciudad,Top área de estudios
0,armenia,literatura
1,barranquilla,sociología
2,bogota,derecho
3,bucaramanga,historia
4,buenaventura,adm. de empresas
5,cali,historia
6,cartagena,derecho
7,cucuta,derecho
8,florencia,derecho
9,girardot,escolar


## Ciudades más importantes

In [21]:
top_ciudades = perfiles_df[perfiles_df['Ciudad'] != 'OTRO']['Ciudad'].value_counts()[:15]

In [22]:
top_ciudades

bogota        21477
tunja          2820
ibague         1934
popayan        1811
neiva          1615
pereira        1175
manizales      1032
ipiales         932
girardot        813
cartagena       632
san andres      566
valledupar      553
florencia       526
leticia         456
quibdo          409
Name: Ciudad, dtype: int64

## Barrios más y menos importantes por ciudad

In [23]:
ciudades_barrios_importantes = {}

N = 10
ciudades_gr = perfiles_df.groupby('Ciudad')
for c, g in ciudades_gr:
    if c in top_ciudades:
        mas = g['Barrio'].value_counts()[:N]
        menos = g['Barrio'].value_counts()[-N:]
        ciudades_barrios_importantes[c] = {}
        ciudades_barrios_importantes[c]['+'] = mas
        ciudades_barrios_importantes[c]['-'] = menos
        
        print('CIUDAD :', c)
        print('USUARIOS EN', c, ':', top_ciudades[c])
        print(mas)
        print('...')
        print(menos)
        print()

CIUDAD : bogota
USUARIOS EN bogota : 21477
chapinero     401
candelaria    282
soledad       252
fontibon      244
cedritos      237
castilla      235
modelia       208
palermo       184
recuerdo      182
centro        169
Name: Barrio, dtype: int64
...
tabio           1
cazuca          1
cardal          1
senderos        1
cajica          1
rosa blanca     1
cañaveralejo    1
guaimaral       1
caney           1
plazuela        1
Name: Barrio, dtype: int64

CIUDAD : cartagena
USUARIOS EN cartagena : 632
crespo            35
manga             32
bocagrande        22
torices           22
san fernando      19
pozon             18
pie de la popa    16
cartagena         15
alpes             13
socorro           13
Name: Barrio, dtype: int64
...
san jose          1
brisas            1
plazuela          1
junin             1
ruby              1
contador          1
villas del sol    1
providencia       1
palmares          1
rubi              1
Name: Barrio, dtype: int64

CIUDAD : florencia
USU

### Top 1 nivel de estudios por barrio

In [24]:
niveles_barrios = perfiles_df[perfiles_df['Nivel Estudio1'] != 'OTRO'] \
    .groupby(['Ciudad', 'Barrio'])['Nivel Estudio1'] \
    .agg(lambda x: x.value_counts().index[0])

In [25]:
top_niveles_barrios_ls = [
    (x, y, z)
    for (x, y), z in niveles_barrios.iteritems()
    if x in ciudades_barrios_importantes and y in ciudades_barrios_importantes[x]['+']
]

In [26]:
top_niveles_barrios_df = pd.DataFrame(top_niveles_barrios_ls, columns=['Ciudad', 'Barrio', 'Top nivel de estudios'])

In [27]:
print_full_df(top_niveles_barrios_df)

,Ciudad,Barrio,Top nivel de estudios
0,bogota,candelaria,profesional
1,bogota,castilla,universitario 2 nivel
2,bogota,cedritos,profesional
3,bogota,centro,profesional
4,bogota,chapinero,profesional
5,bogota,fontibon,universitario 2 nivel
6,bogota,modelia,profesional
7,bogota,palermo,profesional
8,bogota,recuerdo,universitario 1 nivel
9,bogota,soledad,profesional


### Top 1 área de estudios por barrio

In [28]:
areas_barrios = perfiles_df[perfiles_df['Área de Estudio1'] != 'OTRO'] \
    .groupby(['Ciudad', 'Barrio'])['Área de Estudio1'] \
    .agg(lambda x: x.value_counts().index[0])

In [29]:
top_areas_barrios_ls = [
    (x, y, z)
    for (x, y), z in areas_barrios.iteritems()
    if x in ciudades_barrios_importantes and y in ciudades_barrios_importantes[x]['+']
]

In [30]:
top_areas_barrios_df = pd.DataFrame(top_areas_barrios_ls, columns=['Ciudad', 'Barrio', 'Top nivel de estudios'])

In [31]:
print_full_df(top_areas_barrios_df)

,Ciudad,Barrio,Top nivel de estudios
0,bogota,candelaria,derecho
1,bogota,castilla,derecho
2,bogota,cedritos,derecho
3,bogota,centro,derecho
4,bogota,chapinero,derecho
5,bogota,fontibon,derecho
6,bogota,modelia,derecho
7,bogota,palermo,derecho
8,bogota,recuerdo,derecho
9,bogota,soledad,literatura
